# Data Processing

1. Verifica il numero di righe e colonne del dataset
2. Verifica la tipologia di ogni variabile
3. Verifica il numero di valori mancanti per ogni colonna
4. Rimuovi eventualmente le colonne con oltre il 30% di valori mancanti
5. Rimuovi eventualmente le righe con oltre il 25% di valori mancanti
6. Rimuovi tutte le righe dove PRICE è mancante
7. Esegui l'imputazione con valore medio per i restanti valori mancanti quantitativi
8. Esegui la codifica di eventuali variabili qualitative
9. Esegui l'imputazione con moda per i restanti valori mancanti qualitativi
10. Esegui la standardizzazione
11. Salva il nuovo dataframe in un tsv chiamato "housing_clean.tsv"

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/housing_dirty.csv', 
                 index_col = 0)
#1) 
df.shape

(506, 14)

In [3]:
#2)
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 506 entries, 0.0 to 505.0
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    object 
 1   ZN       504 non-null    float64
 2   INDUS    503 non-null    float64
 3   CHAS     506 non-null    object 
 4   NOX      499 non-null    float64
 5   RM       501 non-null    float64
 6   AGE      502 non-null    float64
 7   DIS      501 non-null    float64
 8   RAD      503 non-null    float64
 9   TAX      504 non-null    float64
 10  PTRATIO  501 non-null    float64
 11  B        503 non-null    float64
 12  LSTAT    307 non-null    float64
 13  PRICE    502 non-null    float64
dtypes: float64(12), object(2)
memory usage: 59.3+ KB


In [4]:
#Per le var object essendo di tipo qualitativo posso controllare da cosa sono composte tramite 
print(df['CRIM'].value_counts(),'\n', df['CHAS'].value_counts())

HIGH         130
LOW          127
VERY HIGH    127
MODERATE     122
Name: CRIM, dtype: int64 
 NO     471
YES     35
Name: CHAS, dtype: int64


In [5]:
#3)
df.isna().sum()

CRIM         0
ZN           2
INDUS        3
CHAS         0
NOX          7
RM           5
AGE          4
DIS          5
RAD          3
TAX          2
PTRATIO      5
B            3
LSTAT      199
PRICE        4
dtype: int64

In [6]:
#4) creo valore di thrashold per valori oltre al 30%
thresh = df.shape[0]*.7
#controllo tutte le colonne che eccedono il 70%
df.count()<thresh 

CRIM       False
ZN         False
INDUS      False
CHAS       False
NOX        False
RM         False
AGE        False
DIS        False
RAD        False
TAX        False
PTRATIO    False
B          False
LSTAT       True
PRICE      False
dtype: bool

In [7]:
#elimino le colonne che possiedono valori NaN superiori al 30%
df = df.dropna(thresh = thresh, axis = 1)
df.shape #è presente una colonna in meno 

(506, 13)

In [8]:
#5) Creo un valore di threshold per valori oltre il 25%
df = df.dropna(thresh = int(df.shape[1]*0.75))
df.shape #sono presenti 3 colonne in meno 

(503, 13)

In [9]:
#6) cerco le righe in cui la variabile target price NON è presente (so che sono 4 righe)
df = df.dropna(subset = 'PRICE')
print(df.shape) #infatti sono 4 righe in meno rispetto a prima 
df.isna().sum() #controllo anche mediante isna

(499, 13)


CRIM       0
ZN         1
INDUS      2
CHAS       0
NOX        6
RM         2
AGE        1
DIS        2
RAD        0
TAX        0
PTRATIO    3
B          1
PRICE      0
dtype: int64

In [10]:
#7
cols = df.columns #estraggo tutte le colonne del df
cols = cols.drop(['CRIM', 'CHAS']) #estraggo le colonne con variabili di tipo quantitativo
to_fill = df[cols].mean().round(2) #calcolo il valore medio da inserire nei dati mancanti 
df[cols] = df[cols].fillna(to_fill)
df.isna().sum().sum() #controllo se sono ancora presenti righe vuote con una somma di tutti i valori NaN

0

In [11]:
#8) siccome CRIM possiede delle variabili di tipo qualitativo ordinato 
mapping = {'LOW':1, 'MODERATE':2, 'HIGH': 3, 'VERY HIGH':4}
df['CRIM'] = df['CRIM'].map(mapping)
df['CRIM'].value_counts() #controllo se i valori delle variabili uniche sono cambiate 

1    127
3    127
4    125
2    120
Name: CRIM, dtype: int64

In [12]:
#CHAS possiede solo si e no
map_yn = {'YES':1, 'NO':0}
df['CHAS'] = df['CHAS'].map(map_yn)
df['CHAS'].value_counts()

0    464
1     35
Name: CHAS, dtype: int64

In [14]:
#9 NON è da fare in quanto se si controlla NON esistono valori mancanti qualitativi
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
PRICE      0
dtype: int64

In [16]:
#10 eseguo standardizzazione, ma prima devo togliere la variabile taget 'PRICE'
feautures = df.columns.drop('PRICE') #creo un df con solo le variabili features
df[feautures] = (df[feautures] - df[feautures].mean())/df[feautures].std()
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,PRICE
0.0,-1.335848,0.277489,-1.287063,-0.274372,0.334501,0.472344,-0.115693,-0.297511,-0.983103,-0.667445,-1.476726,0.439108,24.0
1.0,-1.335848,-0.490469,-0.593734,-0.274372,0.025612,0.395632,0.369656,-0.296823,-0.868858,-0.986232,-0.312837,0.439108,21.6
2.0,-1.335848,-0.490469,-0.593734,-0.274372,0.025612,0.776201,-0.260943,-0.296823,-0.868858,-0.986232,-0.312837,0.394661,34.7
3.0,-1.335848,-0.490469,-1.305998,-0.274372,-0.023631,0.683051,-0.802975,-0.295965,-0.754613,-1.104301,0.106164,0.414319,33.4
4.0,-1.335848,-0.490469,-1.305998,-0.274372,-0.023631,0.757272,-0.505389,-0.295965,-0.754613,-1.104301,0.106164,0.439108,36.2


In [17]:
#controllo se la media è circa 0 
df[feautures].mean()

CRIM       5.695733e-17
ZN        -6.763683e-17
INDUS      3.275047e-16
CHAS      -1.423933e-17
NOX        2.207097e-16
RM        -9.255567e-17
AGE       -4.271800e-16
DIS        7.119667e-18
RAD        5.695733e-17
TAX        0.000000e+00
PTRATIO    5.695733e-17
B         -7.119667e-17
dtype: float64

In [18]:
#controllo se la std è circa 1
df[feautures].std()

CRIM       1.0
ZN         1.0
INDUS      1.0
CHAS       1.0
NOX        1.0
RM         1.0
AGE        1.0
DIS        1.0
RAD        1.0
TAX        1.0
PTRATIO    1.0
B          1.0
dtype: float64